In [ ]:
class gallovidia:
    def __init__(self):
        pass
    def record_transaction(self):
        # 1) Read in JSON file or create new JSON if file doesn't exist.
        

In [15]:
import os
import json

path = "C:/Users/PC/OneDrive/GALLOVIDIA INVESTMENTS/Miscellaneous/Transactions/transactions.json"

if os.path.isfile(path=path) == True:
    with open(path, "r") as file:
        transactions = json.load(file)
        print("JSON file detected. Reading.")
else:
    transactions = {}
    with open(path, "w") as file:
        json.dump(transactions, file)
        print("JSON file not detected. A new transactions log JSON has been created.")



JSON file detected. Reading.


In [7]:
import json

# # Load the transaction data from a JSON file
# path = "C:/Users/PC/OneDrive/GALLOVIDIA INVESTMENTS/Miscellaneous/Transactions/transactions.json"

# with open(path, "r") as f:
#     transactions = json.load(f)

# Prompt the user to enter transaction details
transaction = {
    "date": input("Enter transaction date (YYYY-MM-DD): "),
    "account": input("Enter account name: "),
    "security": input("Enter security name: "),
    "value":float(input("Enter value of transaction: ")),
    "quantity": float(input("Enter quantity: ")),
    "price": float(input("Enter price: ")),
    "type": input("Enter transaction type (buy/sell/dividend): "),
    "fx_rate": float(input("Enter FX rate (if applicable): ") or 1.0),
    "fees": float(input("Enter fees (if applicable): ") or 0.0),
}

# Add the new transaction to the list of transactions
# transactions.append(transaction)

# # Save the updated transaction data back to the JSON file
# with open("transactions.json", "w") as f:
#     json.dump(transactions, f)


ValueError: could not convert string to float: ''